# Working On...

In [1]:
import sys
from PyQt6.QtGui import *
from PyQt6.QtWidgets import *
from PyQt6.QtCore import *
import cv2

class GyroSpin_App(QWidget):
    def __init__(self):
        super(GyroSpin_App, self).__init__()
        #self.resize(1000, 1000)

        self.setWindowTitle("Main Menu - GyroSpin App")

        #---   Layout total ---#
        self.LayoutTotal = QHBoxLayout()
        #self.LayoutTotal.setContentsMargins(0, 0, 0, 0)
        
        #---   Layout des commandes ---#

    
        
        self.VBL_Buttons = QVBoxLayout()
        self.HBL_Time = QHBoxLayout()
        self.HBL_Total_Time = QVBoxLayout()
        self.HBL_Check_Boxes = QHBoxLayout()
        
        
        self.Label_Time_Edit = QLabel()
        self.Label_Time_Edit.setText(r'Time interval (ms) : ')
        self.Time_Line_Edit = QLineEdit()
        self.HBL_Time.addWidget(self.Label_Time_Edit)
        self.HBL_Time.addWidget(self.Time_Line_Edit)
        

        
        self.Time_Edit_Button = QPushButton("Edit Time Interval")

        
        self.HBL_Total_Time.addLayout(self.HBL_Time)
        self.HBL_Total_Time.addWidget(self.Time_Edit_Button)
        
        self.VBL_Buttons.addLayout(self.HBL_Total_Time)

        # horizontal spaces
        self.Empty_Space1 = QLabel()
        self.VBL_Buttons.addWidget(self.Empty_Space1)

        self.OpenArduinoWidget = QPushButton("Open Arduino \n control panel")
        self.OpenArduinoWidget.clicked.connect(self.OpenArduinoSettings)
        self.VBL_Buttons.addWidget(self.OpenArduinoWidget)

        self.Empty_Space2 = QLabel()
        self.VBL_Buttons.addWidget(self.Empty_Space2)


        self.OpenCameraWidget = QPushButton("Edit Camera \n settings")
        self.OpenCameraWidget.clicked.connect(self.OpenCameraSettings)
        self.VBL_Buttons.addWidget(self.OpenCameraWidget)
        

        self.Empty_Space3 = QLabel()
        self.VBL_Buttons.addWidget(self.Empty_Space3)



        self.LabelDetection = QLabel("Message for detection...")
        self.VBL_Buttons.addWidget(self.LabelDetection)


        self.Empty_Space4 = QLabel()
        self.VBL_Buttons.addWidget(self.Empty_Space4)

        
        
        self.save_plots = QCheckBox("Save plots")
        #self.save_plots.setText("Save plots")
        self.save_data = QCheckBox("Save data")
        #self.save_data.setText("Save data")
        self.HBL_Check_Boxes.addWidget(self.save_plots)
        self.HBL_Check_Boxes.addWidget(self.save_data)
        self.VBL_Buttons.addLayout(self.HBL_Check_Boxes)

        
        
        
        #---   Layout caméra ---#

        self.VBL = QVBoxLayout()

        self.FeedLabel_cam1 = QLabel()
        self.FeedLabel_cam2 = QLabel()
        self.VBL.addWidget(self.FeedLabel_cam1)
        self.VBL.addWidget(self.FeedLabel_cam2)

        
        self.Cam_Control_Grid = QGridLayout()
        self.Enable_cam1 = QPushButton("Enable Cam1")
        self.Enable_cam2 = QPushButton("Enable Cam2")
        self.Disable_cam1 = QPushButton("Disable Cam1")
        self.Disable_cam2 = QPushButton("Disable Cam2")

        self.Enable_cam1.clicked.connect(self.Set_cam1)
        self.Enable_cam2.clicked.connect(self.Set_cam2)
        self.Disable_cam1.clicked.connect(self.Stop_cam1)
        self.Disable_cam2.clicked.connect(self.Stop_cam2)

        self.Cam_Control_Grid.addWidget(self.Enable_cam1, 0, 0, 1, 1)
        self.Cam_Control_Grid.addWidget(self.Enable_cam2, 0, 1, 1, 1)
        self.Cam_Control_Grid.addWidget(self.Disable_cam1, 1, 0, 1, 1)
        self.Cam_Control_Grid.addWidget(self.Disable_cam2, 1, 1, 1, 1)

        self.VBL.addLayout(self.Cam_Control_Grid)

        self.Worker1 = Worker1()
        self.Worker2 = Worker2()
        
        self.Worker1.start()
        self.Worker2.start()
        
        self.Worker1.ImageUpdate.connect(self.ImageUpdateSlot_1)
        self.Worker2.ImageUpdate.connect(self.ImageUpdateSlot_2)
        
        
        self.LayoutTotal.addLayout(self.VBL)
        self.LayoutTotal.addLayout(self.VBL_Buttons)

        self.setLayout(self.LayoutTotal)
        
        #---
        #self.VBL2.addLayout(self.VHL)
        #self.setLayout(self.VBL_Buttons)

    def ImageUpdateSlot_1(self, Image1):
        self.FeedLabel_cam1.setPixmap(QPixmap.fromImage(Image1))

    def ImageUpdateSlot_2(self, Image2):
        self.FeedLabel_cam2.setPixmap(QPixmap.fromImage(Image2))

    def Stop_cam1(self):
        self.Worker1.stop()
        
    def Stop_cam2(self):
        self.Worker2.stop()

    def Set_cam1(self):
        self.Worker1.set()
        
    def Set_cam2(self):
        self.Worker2.set()

    def OpenArduinoSettings(self):
        self.ArduinoSetting_Window = QMainWindow()
        self.ui = ArduinoSettings_Widget()
        self.ArduinoSetting_Window.show()
    def OpenCameraSettings(self):
        

class Worker1(QThread):
    ImageUpdate = pyqtSignal(QImage)
    def run(self):
        self.ThreadActive = True
        Capture = cv2.VideoCapture(0)
        while self.ThreadActive:
            ret, frame = Capture.read()
            if ret:
                Image1 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                FlippedImage1 = cv2.flip(Image1, 1)
                ConvertToQtFormat1 = QImage(FlippedImage1.data, FlippedImage1.shape[1], FlippedImage1.shape[0], QImage.Format.Format_RGB888)
                Pic1 = ConvertToQtFormat1.scaled(400, 280)
                #Pic = ConvertToQtFormat.scaled(640, 480)
                self.ImageUpdate.emit(Pic1)
    def stop(self):
        self.ThreadActive = False
        self.quit()

    def set(self):
        '''A corriger'''
        self.ThreadActive = True
        self.run()



class Worker2(QThread):
    ImageUpdate = pyqtSignal(QImage)
    def run(self):
        self.ThreadActive = True
        Capture = cv2.VideoCapture(0)
        while self.ThreadActive:
            ret, frame = Capture.read()
            if ret:
                Image2 = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                FlippedImage2 = cv2.flip(Image2, 1)
                ConvertToQtFormat2 = QImage(FlippedImage2.data, FlippedImage2.shape[1], FlippedImage2.shape[0], QImage.Format.Format_RGB888)
                Pic2 = ConvertToQtFormat2.scaled(400, 280)
                self.ImageUpdate.emit(Pic2)
    def stop(self):
        self.ThreadActive = False
        self.quit()

    def set(self):
        '''A corriger'''
        self.ThreadActive = True
        self.run()


SystemExit: 0

/Users/bradaizakaria/Documents/venvL3/envL3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
if __name__ == "__main__":
    App = QApplication(sys.argv)
    Root = GyroSpin_App()
    Root.show()
    sys.exit(App.exec())

In [ ]:
import sys
from PyQt6.QtGui import *
from PyQt6.QtWidgets import *
from PyQt6.QtCore import *
import cv2

class MainWindow(QWidget):
    def __init__(self):
        super(MainWindow, self).__init__()
        #self.resize(1000, 1000)

        #---   Layout total ---#
        self.LayoutTotal = QHBoxLayout()
        #self.LayoutTotal.setContentsMargins(0, 0, 0, 0)
        
        #---   Layout des commandes ---#

    
        
        self.VBL_Buttons = QVBoxLayout()
        self.VHL_Time = QHBoxLayout()
        self.VHL_Total_Time = QVBoxLayout()
        self.VHL_Check_Boxes = QHBoxLayout()
        
        
        self.Label_Time_Edit = QLabel()
        self.Label_Time_Edit.setText("Time interval : ")
        self.Time_Line_Edit = QLineEdit()
        self.VHL_Time.addWidget(self.Label_Time_Edit)
        self.VHL_Time.addWidget(self.Time_Line_Edit)
        #self.VBL_Buttons.addLayout(self.VHL_Time)
        

        
        self.Time_Edit_Button = QPushButton("Edit Time Interval")
        #self.VBL_Buttons.addWidget(self.Time_Edit_Button)

        
        self.VHL_Total_Time.addLayout(self.VHL_Time)
        self.VHL_Total_Time.addWidget(self.Time_Edit_Button)
        
        self.VBL_Buttons.addLayout(self.VHL_Total_Time)

        # 2 horizontal spaces
        self.Empty_Space1 = QLabel()
        #self.Empty_Space2 = QLabel()
        self.VBL_Buttons.addWidget(self.Empty_Space1)
        #self.VBL_Buttons.addWidget(self.Empty_Space2)
        #
        
        self.save_plots = QCheckBox("Save plots")
        #self.save_plots.setText("Save plots")
        self.save_data = QCheckBox("Save data")
        #self.save_data.setText("Save data")
        self.VHL_Check_Boxes.addWidget(self.save_plots)
        self.VHL_Check_Boxes.addWidget(self.save_data)
        self.VBL_Buttons.addLayout(self.VHL_Check_Boxes)

        
        
        
        #---   Layout caméra ---#

        self.VBL = QVBoxLayout()
        

        self.FeedLabel = QLabel()
        self.VBL.addWidget(self.FeedLabel)
        self.CancelBTN = QPushButton("Cancel")
        self.CancelBTN.clicked.connect(self.CancelFeed)
        self.Empty_Space3 = QLabel()
        self.VBL.addWidget(self.Empty_Space3)
        self.VBL.addWidget(self.CancelBTN)

        self.Worker1 = Worker1()

        self.Worker1.start()
        self.Worker1.ImageUpdate.connect(self.ImageUpdateSlot)
        
        
        
        #layout.setContentsMargins(0, 0, 0, 0)

        #self.setLayout(self.VBL)

        self.LayoutTotal.addLayout(self.VBL)
        self.LayoutTotal.addLayout(self.VBL_Buttons)

        self.setLayout(self.LayoutTotal)
        
        #---
        #self.VBL2.addLayout(self.VHL)
        #self.setLayout(self.VBL_Buttons)

    def ImageUpdateSlot(self, Image):
        self.FeedLabel.setPixmap(QPixmap.fromImage(Image))

    def CancelFeed(self):
        self.Worker1.stop()

class Worker1(QThread):
    ImageUpdate = pyqtSignal(QImage)
    def run(self):
        self.ThreadActive = True
        Capture = cv2.VideoCapture(0)
        while self.ThreadActive:
            ret, frame = Capture.read()
            if ret:
                Image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                FlippedImage = cv2.flip(Image, 1)
                ConvertToQtFormat = QImage(FlippedImage.data, FlippedImage.shape[1], FlippedImage.shape[0], QImage.Format.Format_RGB888)
                Pic = ConvertToQtFormat.scaled(640, 500)
                #Pic = ConvertToQtFormat.scaled(640, 480)
                self.ImageUpdate.emit(Pic)
    def stop(self):
        self.ThreadActive = False
        self.quit()



if __name__ == "__main__":
    App = QApplication(sys.argv)
    Root = MainWindow()
    Root.show()
    sys.exit(App.exec())